Student's Name and Email Address

Boise State University, Department of Chemistry and Biochemistry

## CHEM 324: PChem Lab {-}
# Worksheet 4: Computational Understanding of Molecular Vibrations {-}

In [ ]:
# @title PySCF Setup { display-mode: "form" }
# Import the main components of PySCF used in this worksheet
!pip install pyscf
!pip install pyberny
!pip install pyscf\[geomopt\]
!pip install git+https://github.com/pyscf/properties
from pyscf import gto, scf, dft, mp, cc
from pyscf.geomopt.berny_solver import optimize
from pyscf.hessian import rks, thermo 
from pyscf.prop.infrared import rks as rks_ir
from pyscf.prop.infrared import rhf as rhf_ir
#
from scipy.constants import physical_constants # we will need these for units conversion

In [ ]:
# @title Notebook Setup { display-mode: "form" }
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import time
# Load the google drive with your files
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
base_path = '/content/drive/MyDrive/'

In [ ]:
# @title Set Local Path { display-mode: "form" }
# The following needs to be the path of the folder with all your collected data in .csv format
local_path="Colab Notebooks/CompVib_Data/" # @param {type:"string"}
path = base_path+local_path

In [ ]:
# @title Install and load RDKit, CirPy, and Py3DMol { display-mode: "form" }
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Draw
!pip install cirpy
import cirpy
! pip install py3Dmol
import py3Dmol

In [ ]:
# @title Utilities { display-mode: "form" }
def quadratic_vertex_fit(x, y):
    """Fit y = ax^2 + bx + c through 3 points; return vertex (x0, y0) and coefficients."""
    x = np.asarray(x)
    y = np.asarray(y)
    A = np.vstack([x**2, x, np.ones_like(x)]).T
    a, b, c = np.linalg.solve(A, y)
    x0 = -b / (2*a)
    y0 = a*x0**2 + b*x0 + c
    return x0, y0, a, b, c

def quadratic_fit_3pt(x, y):
    """Fit y = ax^2 + bx + c through 3 points and return a,b,c and vertex."""
    x = np.asarray(x)
    y = np.asarray(y)
    A = np.vstack([x**2, x, np.ones_like(x)]).T
    a, b, c = np.linalg.solve(A, y)
    x0 = -b / (2*a)
    y0 = a*x0**2 + b*x0 + c
    return a, b, c, x0, y0

def plot_pes_with_centered_quadratic(distances_A, energies_eh, show_fit=False, title="Diatomic PES"):
    x = np.asarray(distances_A)
    E = np.asarray(energies_eh)

    i0 = int(np.argmin(E))
    if i0 == 0 or i0 == len(E) - 1:
        raise ValueError("Minimum is at boundary; expand dmin/dmax or increase resolution.")

    # --- 3-point quadratic fit in standard form ---
    x3 = x[i0-1:i0+2]
    E3 = E[i0-1:i0+2]
    A = np.vstack([x3**2, x3, np.ones_like(x3)]).T
    a_std, b_std, c_std = np.linalg.solve(A, E3)

    # Convert to centered form: E = E0 + a (R - Req)^2
    Req = -b_std / (2*a_std)
    E0 = a_std*Req**2 + b_std*Req + c_std
    a = 2 * a_std  # curvature parameter in Eh / Å^2

    # Convert to relative energy (kJ/mol) for plotting
    Eh_to_kJmol = 2625.499638
    dE_kJmol = (E - E.min()) * Eh_to_kJmol

    # Local quadratic curve
    x_fit = np.linspace(x.min(), x.max(), 250)
    E_fit = E0 + 0.5*a*(x_fit - Req)**2
    dE_fit_kJmol = (E_fit - E.min()) * Eh_to_kJmol

    # ---- Plot ----
    plt.figure(figsize=(8.5, 5.2))
    plt.plot(x, dE_kJmol, marker='o', markersize=4, linewidth=1.8, label='PES')
    if show_fit:
        plt.plot(x_fit, dE_fit_kJmol, linewidth=2.2, label='Quadratic Fit')

    # Mark grid minimum
    plt.scatter([x[i0]], [(E[i0]-E.min())*Eh_to_kJmol], s=90, marker='x', color='red', label='Grid Minimum')

    # Mark quadratic minimum
    plt.axvline(Req, linestyle='--', linewidth=1.6)
    plt.scatter([Req], [(E0-E.min())*Eh_to_kJmol], s=70, marker='D', color='green', label='Quadratic Minimum')

    plt.xlabel("Bond distance $R$ (Å)")
    plt.ylabel(r"Relative energy $\Delta E$ (kJ/mol)")
    plt.title(title)
    plt.grid(True, alpha=0.25)
    plt.legend(loc='upper right')

    # Info box
    info = (
        f"Grid min R = {x[i0]:.6f} Å\n"
        f"Quad  Re  = {Req:.6f} Å\n\n"
        "Fit form:\n"
        "E(R) = E(Re) + 0.5 * a (R - Re)^2\n\n"
        f"a = {a:.6e} Eh/Å²"
    )

    plt.text(
        0.02, 0.98, info,
        transform=plt.gca().transAxes,
        verticalalignment='top',
        bbox=dict(facecolor='white', edgecolor='black')
    )

    plt.tight_layout()
    plt.show()

    print("Centered quadratic fit:")
    print("E(R) = E(Re) + 0.5 * a (R - Re)^2")
    print(f"Re = {Req:.10f} Å")
    print(f"a  = {a:.10e} Eh/Å^2")

    return a, Req, E0

def plot_hessian(mol,H):
    natm = H.shape[0]
    # Flatten (natm,3,natm,3) -> (3N, 3N)
    Hcart = np.transpose(H, (0, 2, 1, 3)).reshape(3*natm, 3*natm)
    Hsym = 0.5 * (Hcart + Hcart.T)
    Hsym[np.abs(Hsym) < 1e-8] = 0

    # Labels: O1x, O1y, O1z, H2x, ...
    axes = ("x", "y", "z")
    labels = [f"{mol.atom_symbol(i)}{i}{ax}"
          for i in range(natm) for ax in axes]

    fig, ax = plt.subplots(figsize=(6, 5))
    v = np.max(np.abs(Hsym))
    im = ax.imshow(Hsym, cmap="coolwarm", origin="lower", vmin=-v, vmax=v)
    fig.colorbar(im, ax=ax, label="H (Eh/Bohr² or Eh/(Bohr·rad))")

    ax.set_xticks(range(3*natm)); ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(range(3*natm)); ax.set_yticklabels(labels)

    ax.set_title("Molecular Hessian")
    ax.set_xlabel("Cartesian coordinate")
    ax.set_ylabel("Cartesian coordinate")
    plt.tight_layout()
    plt.show()

def plot_dynamical_matrix(mol,H,mass):
    #
    atom_coords = mol.atom_coords()
    mass_center = np.einsum('z,zx->x', mass, atom_coords) / mass.sum()
    atom_coords = atom_coords - mass_center
    natm = atom_coords.shape[0]
    D = np.einsum('pqxy,p,q->pqxy', H, mass**-.5, mass**-.5)
    #
    natm = D.shape[0]
    # Flatten (natm,3,natm,3) -> (3N, 3N)
    Dcart = np.transpose(D, (0, 2, 1, 3)).reshape(3*natm, 3*natm)
    Dsym = 0.5 * (Dcart + Dcart.T)
    Dsym[np.abs(Dsym) < 1e-8] = 0

    # Labels: O1x, O1y, O1z, H2x, ...
    axes = ("x", "y", "z")
    labels = [f"{mol.atom_symbol(i)}{i}{ax}"
          for i in range(natm) for ax in axes]

    fig, ax = plt.subplots(figsize=(6, 5))
    v = np.max(np.abs(Dsym))
    im = ax.imshow(Dsym, cmap="coolwarm", origin="lower", vmin=-v, vmax=v)
    fig.colorbar(im, ax=ax, label="D (Eh/Bohr² or Eh/(Bohr·rad))")

    ax.set_xticks(range(3*natm)); ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(range(3*natm)); ax.set_yticklabels(labels)

    ax.set_title("Molecular Dynamical Matrix")
    ax.set_xlabel("Cartesian coordinate")
    ax.set_ylabel("Cartesian coordinate")
    plt.tight_layout()
    plt.show()

def get_cartesian_mode(mol, analysis, masses, mode_index):
    """
    Returns Cartesian displacement vectors for a selected mode.
    mode_index: index in sorted freq list
    """
    modes = analysis["norm_mode"]      # shape (3N, 3N)
    # Select mode column
    q = modes[mode_index]
    # Divide by sqrt(mass)
    disp = q / np.sqrt(masses)[:, None]
    return disp

def make_vibration_frames(mol, disp, amplitude=0.5, nframes=60, coords_in_bohr=True, disp_in_bohr=True):
    bohr_to_ang = 0.529177210903
    coords = mol.atom_coords()
    if coords_in_bohr:
        coords = coords * bohr_to_ang  # now Å
    # Make sure disp is in Å too
    dispA = disp
    if disp_in_bohr:
        dispA = disp * bohr_to_ang
    atoms = [mol.atom_symbol(i) for i in range(mol.natm)]
    frames = []
    # endpoint=False avoids duplicating the first frame at 2π
    for t in np.linspace(0, 2*np.pi, nframes, endpoint=False):
        factor = amplitude * np.sin(t)
        new_coords = coords + factor * dispA
        xyz = [f"{mol.natm}", f"t={t:.6f}"]
        for sym, (x, y, z) in zip(atoms, new_coords):
            xyz.append(f"{sym} {x:.6f} {y:.6f} {z:.6f}")
        frames.append("\n".join(xyz))
    return "\n".join(frames) + "\n"

def show_vibration(frames):
    view = py3Dmol.view(width=500, height=400)
    view.addModelsAsFrames(frames, 'xyz')
    view.setStyle({'stick': {'radius': 0.20}, 'sphere': {'scale': 0.35}})
    view.zoomTo()
    view.animate({'loop': 'forward', 'interval': 30})  # smaller interval = smoother
    view.show()
    return view

def read_jdx(file, len_drop=2):
    drop_idxs = []
    with open(file) as f:
        for i, line in enumerate(f):
            if line.lstrip().startswith('#'):
                continue
            drop_idxs.append(i)
            if len(drop_idxs) == len_drop:
                break
    df = pd.read_csv(file, comment="#", sep=r"\s+", skiprows=drop_idxs, header=None, names=["wavenumber", "transmittance"], usecols=[0, 1])
    return df

## Task 1: Understanding the Harmonic Approximation {-}

**Introduction** 

You probably have studied vibrations for the first time in your intro physics course when discussing the motion of a body attached to a spring. In that context, the spring is an object that provides a force opposite and proportional to the displacement from equilibrium: 
$$f^{harmonic} = -k(x-x_{eq}) $$
This harmonic force turns out to be conservative, with an associated elastic potential energy that is quadratic in the displacement
$$U^{harmonic} = \frac{1}{2}k(x-x_{eq})^2$$

Now, molecules are not made of springs, but it turns out that thanks to Taylor expansion we can approximate the motion of any system close to equilibrium using an harmonic approximation:
$$ E^{tot}(x) \approx E^{tot}(x_{eq}) + \left.\frac{dE^{tot}}{dx}\right|_{x_{eq}} \cdot(x-x_{eq}) + \frac{1}{2}\left.\frac{d^2E^{tot}}{dx^2}\right|_{x_{eq}} \cdot(x-x_{eq})^2$$
In the righthand side, the first term is a constant (the energy of the equilibrium configuration), the second term vanishes because of our definition of equilibrium (minimum of the potential energy surface, i.e. zero derivative of the Energy), so we are only left with the quadratic term, that is identican to the harmonic potential above and corresponds to a spring with a force contant of 
$$k = \left.\frac{d^2E^{tot}}{dx^2}\right|_{x_{eq}}$$

Following the classical derivation, knowing the force constant of a spring we can obtain its vibrational frequency as

$$ \omega = \sqrt{\frac{k}{m}}$$

or (for a result in $cm^{-1}$)

$$ \nu = \frac{1}{2\pi c}\sqrt{\frac{k}{m}}$$

In the following, you will use PySCF to compute the potential energy surface (PES) of a diatomic molecule as a function of the separation between the two atoms. For a diatomic molecule we can show that there is a single vibrational motion and that the mass associated to such a motion is the reduced mass $\mu$ of the molecule
$$\frac{1}{\mu} = \frac{1}{m_1} + \frac{1}{m_2}$$
where $m_1$ and $m_2$ are the masses of the two atoms. 

**Experiments** 




For this task, you will need to select two different diatomic molecules and compute its associated vibrational frequency using the harmonic approximation and your choice of model chemistry. One of the two molecules needs to contain at least one hydrogen atom. 

Perform the following steps: 
1. Do a first scan of the PES of the molecule over a reasonably large range of distances
2. Using the results of the first scan, do a second scan of the PES close to the equilibrium distance
3. Use the harmonic approximation near the equilbrium position to compute the vibrational frequency (in $cm^{-1}$) of the molecule for the most common isotopes of the two atoms involved
4. Compute the vibrational frequencies of all other possible isotopologues (i.e. molecules composed by other isotopes) of the chosen molecules
5. Scale your computed frequencies using the scaling factor from the [NIST database](https://cccbdb.nist.gov/vibscalejustx.asp) that corresponds to your model chemistry.
6. Compare your predictions with the experimental values for the fundamental vibrations of diatomic molecules from the corresponding [NIST experimental database of diatomic molecules](https://cccbdb.nist.gov/expdiatomicsx.asp)

Try at least two model chemistry, report your results (equilibrium position, equilibrium energy, quadratic coefficient, vibrational frequency, etc.) and comment on the following points:
* How close to an harmonic potential is the PES of your system when you look at it during your first scan?
* How reasonable is the harmonic approximation when you only focus at the PES of your system for energies that close to the minimum and comparable to the thermal energy?
* How close to the experimental values are your predictions? Which model chemistry works better? 
* Is changing hydrogen to deuterium affecting the computed frequencies more or less than other isotopic substitutions?
* How important is the model chemistry to predict the isotopologue shifts? 

## Task 2: Vibrational Modes of Formic Acid {-}

**Target**

For a molecule with N atoms in 3 dimensions, you should expect to find 3N-6 vibrational modes. These modes will involve displacements from the equilibrium values of all the cartesian coordinates of all the atoms in the molecule. However, in order to compute these modes, most of the steps are identical to what you have done in the previous task, namely: 
1. Generate the system initial configuration
2. Optimize the geometry of the molecule (atom positions) with the chosen model chemistry
3. Compute the Hessian matrix (i.e. the second derivatives of the energy wrt atomic displacements)
4. Divide by the masses of the atoms to generate the dynamical matrix
5. Diagonalize the dynamical matrix to compute the vibrational frequencies and modes
6. Compute IR intensities and the IR absorption spectrum

**Experiments** 

Consider a formic acid molecule. Perform the 6 steps above to compute the vibrational modes and the IR spectrum of the molecule in vacuum using at least two different model chemistries. Comment on your results and make sure to address the following points:
* Which types of modes have higher frequency? Are these modes more or less delocalized over the entire molecule than the lower-frequency ones? 
* Use the NIST scaling factors to rescale the computed vibrational frequencies. Is the order of the modes affected by the model chemistry you adopted? How large are the differences between different calculations? 
* Compare your results with experimental vibrational frequencies, which model (accounting for uniform scaling) is more accurate?

## Task 3: Isotopologues of Formic Acid {-}

**Target**

In this task we want to quantify the effect of isotopic substitutions on the vibrational frequencies of the formic acid molecule. Steps 1-3 of the calculations are identical to the previous step. However, before step 4 you will need to manually adjust the mass of the atom you want to substitute. 

**Experiments** 

Use the model chemistry that you found more accurate in the previous task. For each atom of the formic acid molecule, consider isotopic substitutions with its second most common isotope ($H^1\rightarrow D^2$,  $C^{12}\rightarrow C^{13}$, $O^{16}\rightarrow O^{18}$). Recompute the vibrational modes and the IR spectrum. Given that the molecule is composed of 5 atoms and you are only asked to consider one isotopic substitution at a time, in this task you will compute 5 sets of vibrational frequencies and IR specra. Comment on your results and make sure to address the following points:
* For each substitution report the modes that are affected.
* Which is the largest frequency change that you observe? 
* Are there isotopic substitutions that affect significantly more than one peak? 

## Task 4: Finding the Isotopologue {-}

**Target**

In this task you will be randomly assigned the spectra of one of the possible isotopologues of ethanol. You will be allowed to run no more than 4 calculationos to predict which specific isotopologue you have. 

**Experiments** 

Use the following model chemistry (DFT with the PBE functional and the def2-TZVPP basis set) with a scaling factor of 0.985 (from [Truhlar et al.](https://comp.chem.umn.edu/freqscale/210722_Database_of_Freq_Scale_Factors_v5.pdf)) to optimize the geometry and compute the vibrational frequencies and IR intensities of ethanol. Adjust the isotopes of the hydrogen atoms (changing to deuterium) to figure out which is the unkonwn isotopologue assigned to you. Note that any hydrogen atom can be replaced by a deuterium atom. You can only simulate up to 4 different versions of the molecule, including the non-isotopically substituted one. Report your tests and the final identify of your isotopologue, commenting on your chosen strategy.


In [ ]:
# This cell is used to allow Google Colab to install the tools to convert the notebook to a pdf file
# Un-comment the following lines when you are ready to export the pdf 
#!apt-get update > /dev/null
#!apt-get install texlive texlive-xetex texlive-latex-extra pandoc > /dev/null

In [ ]:
# Use this command to convert the finished worksheet into a pdf 
# NOTE : you may want to change the path of the file, if you are working in a different folder of the Google Drive
# NOTE 2: you need to make sure the google drive is mounted (the second cell at the top) and the path to the file below is correct before running this command
#!jupyter nbconvert --no-input --to PDF "/content/drive/MyDrive/Colab Notebooks/CompVib_Worksheet.ipynb"